In [ ]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [ ]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", "3.0.0")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
dataset


DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [ ]:
# Print 10 articles and highlights
for i in range(10):
    article = dataset['train'][i]['article']
    highlights = dataset['train'][i]['highlights']
    print(f"Article {i+1}:")
    print(f"Article: {article}")
    print(f"Highlights: {highlights}")


Article 1:
Article: LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office c

In [ ]:
event_topics = ['politics', 'crime', 'sports', 'travel', 'health']

In [ ]:
def extract_event_paragraphs(paragraphs, topic):
    keywords = {'politics': ['government', 'election', 'policy'],
                'crime': ['murder', 'theft', 'arrest'],
                'sports': ['game', 'team', 'player'],
                'travel': ['trip', 'vacation', 'destination'],
                'health': ['disease', 'medicine', 'hospital']}
    relevant_paragraphs = []
    for paragraph in paragraphs:
        article_text = paragraph['article']
        for keyword in keywords[topic]:
            if keyword in article_text:
                relevant_paragraphs.append(paragraph)
                break
    return relevant_paragraphs

for topic in event_topics:
    relevant_paragraphs = extract_event_paragraphs(dataset['train'], topic)
    print(f"Extracted {len(relevant_paragraphs)} paragraphs related to {topic}")

Extracted 66029 paragraphs related to politics
Extracted 51148 paragraphs related to crime
Extracted 86025 paragraphs related to sports
Extracted 36213 paragraphs related to travel
Extracted 44131 paragraphs related to health


In [ ]:
import spacy

nlp = spacy.load('en_core_web_sm')

def extract_event_components(paragraph):
    doc = nlp(paragraph['article'])
    entities = [(entity.text, entity.label_) for entity in doc.ents]
    objects = [entity for entity in entities if entity[1] in ['OBJECT', 'PRODUCT']]
    agents = [entity for entity in entities if entity[1] in ['PERSON', 'ORGANIZATION']]
    timespan = [entity for entity in entities if entity[1] == 'DATE']
    geolocation_span = [entity for entity in entities if entity[1] == 'GPE']
    background = [token.text for token in doc if token.pos_ == 'NOUN' and token not in objects + agents]
    return {
        'background': background,
        'objects': objects,
        'agents': agents,
        'timespan': timespan,
        'geolocation_span': geolocation_span
    }

for topic in event_topics:
    relevant_paragraphs = extract_event_paragraphs(dataset['train'], topic)
    for paragraph in relevant_paragraphs:
        event_components = extract_event_components(paragraph)
        print(f"Event components for paragraph: {event_components}")

Event components for paragraph: {'background': ['quarterback', 'pay', 'officials', 'league', 'star', 'court', 'judge', 'say', 'plea', 'deal', 'ring', 'part', 'plea', 'agreement', 'prosecutors', 'conduct', 'team', 'fans', 'actions', 'letter', 'status', 'suspension', 'proceedings', 'papers', 'court', 'co', '-', 'conspirators', 'dogs', 'Falcons', 'owner', 'admissions', 'actions', 'suspension', 'statement', 'conduct', 'reputation', 'statement', 'suspension', '»', 'claims', 'remedies', 'signing', 'bonus', 'year', 'contract', 'count', 'Conspiracy', 'Aid', 'Activities', 'plea', 'agreement', 'charge', 'years', 'prison', 'fine', 'restitution', 'assessment', 'years', 'release', 'plea', 'deal', 'prosecutors', 'end', 'sentencing', 'guidelines', 'defendant', 'defendant', 'fact', 'offense', 'plea', 'agreement', 'summary', 'facts', 'agreement', 'pit', 'bulls', 'property', 'training', 'dogs', 'statement', 'fights', 'money', 'operations', 'gambling', 'monies', 'summary', 'facts', 'Gambling', 'wins', 'c

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-32d5a8afc493>", line 24, in <cell line: 21>
    event_components = extract_event_components(paragraph)
  File "<ipython-input-9-32d5a8afc493>", line 6, in extract_event_components
    doc = nlp(paragraph['article'])
  File "/usr/local/lib/python3.10/dist-packages/spacy/language.py", line 1049, in __call__
    doc = proc(doc, **component_cfg.get(name, {}))  # type: ignore[call-arg]
  File "spacy/pipeline/trainable_pipe.pyx", line 52, in spacy.pipeline.trainable_pipe.TrainablePipe.__call__
  File "spacy/pipeline/transition_parser.pyx", line 264, in spacy.pipeline.transition_parser.Parser.predict
  File "spacy/pipeline/transition_parser.pyx", line 285, in spacy.pipeline.transition_parser.Parser.greedy_parse
  File "/usr/local/lib/python3.10/dist-packages/thinc/model.py", l


KeyboardInterrupt

